# Hey sweetheart!
## Let's segment some Stuff!


#### For a smooth ride, use a kernel from a nice clean virtualenv with (the right) TF and other dependencies installed: 

Follow [THIS](http://anbasile.github.io/programming/2017/06/25/jupyter-venv/) and [THIS](https://www.tensorflow.org/install/install_linux), 
or try the following recipe, after editing ```requirements``` to have the right version of tensorflow[-gpu] ,   <br>corresponding to your HW & CUDA setup (yep, it's not cross-compatible. Sad.)
```
    mkdir ~/envs && mkdir ~/envs/fresh4seg
    virtualenv ~/envs/fresh4seg -p python2  # w.o. --system-packages. let's have it clean
    source ~/envs/fresh4seg/bin/activate
    which pip                      # verify it's the venvs' (not system) pip
    pip install -r requirements
    which ipython                  # verify it's the venvs' (not system) ipython
    ipython kernel install --user --name=fresh4seg
    deactivate
```
Run the system (not venv) ***jupyter notebook [flags]***, then in notebook Kernel-> Change kernel -> fresh4seg

In [ ]:
import tensorflow as tf
print tf.__version__

#### Download a pre-trained model

In [3]:
!mkdir tmp
!cd tmp
!wget https://github.com/hailotech/seg-mentor/releases/download/v0.5/inception_v1_fcn16.tgz
!tar -xzvf inception_v1_fcn16.tgz
!cd ..

mkdir: cannot create directory ‘tmp’: File exists
--2018-05-02 16:18:06--  https://github.com/hailotech/seg-mentor/releases/download/v0.5/inception_v1_fcn16.tgz
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2018-05-02 16:18:06 ERROR 404: Not Found.

tmp/Apr24_inception_v1__1/fcn.ckpt.data-00000-of-00001
tmp/Apr24_inception_v1__1/fcn.ckpt.index
tmp/Apr24_inception_v1__1/fcn.ckpt.meta
tmp/Apr24_inception_v1__1/runargs


#### Download some media

In [ ]:
!wget https://github.com/hailotech/seg-mentor/releases/download/v0.5/perfectparking.mp4    

In [14]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="perfectparking.mp4" type="video/mp4">
</video></div>

#### Now let's prepare and run the model:

In [2]:
from fcn_test import *
import json 

traindir = 'tmp/Apr24_inception_v1__1/'
checkpoint = traindir+'fcn.ckpt' 
pixels = 512
cfg = json.load(open(traindir+'/runargs'))

fcn_builder = fcn_arch.FcnArch(number_of_classes=number_of_classes, is_training=False, net=cfg['basenet'],
                               trainable_upsampling=cfg['trainable_upsampling'], fcn16=cfg['fcn16'])

In [3]:
tf.reset_default_graph()
fcnfunc_img2labels = lambda img: tf.argmax(fcn_builder.build_net(img), dimension=3)
segment_movie(fcnfunc_img2labels, checkpoint, 'perfectparking.mp4', pixels)

Instructions for updating:
Use the `axis` argument instead
INFO:tensorflow:Restoring parameters from tmp/Apr24_inception_v1__1/fcn.ckpt
[MoviePy] >>>> Building video perfectparking_segmented.mp4
[MoviePy] Writing video perfectparking_segmented.mp4


100%|██████████| 43/43 [00:02<00:00, 19.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: perfectparking_segmented.mp4 



This should give you a gist of how the code's high level part works. You'll have to dive into the files for further details, I guess...
#### Side note: same functionality available from command line as:
``` source ~/envs/fresh4seg/bin/activate
python fcn_test.py --traindir=tmp/Apr24_inception_v1__1 --movie=perfectparking.mp4
```

#### OK let's see what we got:

In [5]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="perfectparking_segmented.mp4" type="video/mp4">
</video></div>

Well, you all probably want a smart API then movie.mp4 -> movie_segmented.mp4..
<br> I guess you'll need to dive a bit into the code for that. As russian proverb goes, "without work you can't even get a small fish out of the pond"

#### Let's segment a single image now, with legends and stuff:

In [6]:
from fcn_test import *
%matplotlib inline

tf.reset_default_graph()
fcnfunc_img2labels = lambda img: tf.argmax(fcn_builder.build_net(img), dimension=3)

iterator = single_image_feed('/home/finkel/media/uber1.jpg', pixels)
X_as_in_visualize_each_Xth_seg = 1
image, annotation = iterator.get_next()
predictions = fcnfunc_img2labels(tf.expand_dims(image, axis=0))
test(image, annotation, predictions, checkpoint, iterator)

Instructions for updating:
Use the `axis` argument instead


ValueError: Tensor conversion requested dtype float32 for Tensor with dtype int32: 'Tensor("Cast_1:0", shape=(1, 512, 512, 1), dtype=int32)'

In [13]:
# TODO!